# Week 03 Assignment Covid

Welcome to **week three** of this course programming 1. You will learn about combining data with pandas and numpy and you will learn to visualize with bokeh. Concretely, you will preprocess the partly Synthetic Covid data in an appropiate format in order to conduct statistical and visual analysis. Learning outcomes:

- Load a tabular dataset
- Inspect the dataset for quality and metadata information
- Combine data from several tables into one dataframe
- Subselect specific data from dataframes
- Reshape the dataset into a format suitable for visual and statistical analysis
- Visualize data using bokeh 
- Use widgets to make the plot interactive (optional)
- Use geomap to plot locations (optional)


Your job is to **visualize the lab values taken for COVID-19 patients of survived versus not survived patients**. 

The assignment consists of 6 parts:

- [part 1: load the data](#0)
     - [Exercise 1.1](#ex-11)
- [part 2: data wrangling](#1)
     - [Exercise 2.1](#ex-21)
- [part 3: more wrangling](#2)
     - [Exercise 3.1](#ex-31)
- [part 4: plot the data](#3)
     - [Exercise 4.1](#ex-41)
- [part 5: plot patient location](#5)
     - [Exercise 5.1](#ex-51)


Part 1 and 4 are mandatory, part 5 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


## About the data

The data is generated by Synthea's COVID-19 module. The data was constructed using three peer-reviewed publications published in the early stages of the global pandemic, when less was known, along with emerging resources, data, publications, and clinical knowledge. The simulation outputs synthetic Electronic Health Records (EHR), including the daily consumption of Personal Protective Equipment (PPE) and other medical devices and supplies. For this assignment the `conditions`, `patients`, `observations`, `careplans` and `encounters` table will be used. The Data is stored in separate tables to avoid redundancy, with as a concequence that tables need to be combined and reorganized in dataframes for analysing purpose.

Source: Walonoski J, Klaus S, Granger E, Hall D, Gregorowicz A, Neyarapally G, Watson A, Eastman J. Synthea™ Novel coronavirus (COVID-19) model and synthetic data set. Intelligence-Based Medicine. 2020 Nov;1:100007. https://doi.org/10.1016/j.ibmed.2020.100007

Please <a href = "https://storage.googleapis.com/synthea-public/10k_synthea_covid19_csv.zip">download</a> the data

#### Covid Patients
Patients are considered Covid patients if they are identified with `CODE` `840539006`


#### Survivors
Patients that had covid and where tested negative after isolation have tested code `94531-1`,  SARS-CoV-2 RNA Pnl Resp NAA+probe (covid-sars test) + a value of `Not detected (qualifier value)`. These patients are considered to be survived covid patients. 

#### Non-Survivors
Patients that did not survived Covid have a `DEATHDATE` which is not null. 


#### Lab values  COVID-19 patients

Patients are monitored for blood and heart conditions once they are admitted in Hospital or under treatment. The lab values of interest are as follow: 

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction


---

<a name='0'></a>
## Part 1: Load the data (20 pt)

Instructions: Load the data of the following files

- conditions.csv
- patients.csv
- observations.csv
- careplans.csv
- encounters.csv

Get yourself familiar with the data. Create some meaningful overviews. Answer the following questions

1. How many patients are there
2. How many covid-patients are there
3. How many patients do have a 'Hospital admission for isolation' encounter
    
<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>use a unique dataframe for each file, use a meaningful name</li>
    <li>pandas.read_csv() method can be used to read a csv file</li>
    <li>pandas.DataFrame.head() method is often used to inspect the dataframe</li>
    <li>.unique() returns a list of unique values of a column</li>
</ul>
</details>

<a name='ex-11'></a>
### 1.1 Code your solution

In [1]:
#YOUR CODE HERE
import pandas as pd
import numpy as np
df_condition=pd.read_csv(r"D:\programming1\10k_synthea_covid19_csv\conditions.csv")
df_patient=pd.read_csv(r"D:\programming1\10k_synthea_covid19_csv\patients.csv")
df_observ=pd.read_csv(r"D:\programming1\10k_synthea_covid19_csv\observations.csv")
df_careplan=pd.read_csv(r"D:\programming1\10k_synthea_covid19_csv\careplans.csv")
df_encounter=pd.read_csv(r"D:\programming1\10k_synthea_covid19_csv\encounters.csv")
print('There are ',len(pd.DataFrame(df_patient.groupby('Id'))),' patients in total.')
print('There are ',len(pd.DataFrame(df_condition[df_condition['CODE']==840539006])),' covid patients')
df_encounter['DESCRIPTION'].value_counts()
print('There are',len(pd.DataFrame(df_encounter[df_encounter['DESCRIPTION']=='Hospital admission for isolation (procedure)'])),'admitted patients')
print(df_patient['DEATHDATE'].count(),'patients died')



There are  12352  patients in total.
There are  8820  covid patients
There are 1867 admitted patients
2352 patients died


### 1.2 Test your solution
The following function needs to be called. You can use this as a test. There are however more meaningful overviews 
you can create. 

In [2]:
def part1(num_pat, num_cov, num_admitted, num_died):
    print(f'There are {num_pat} patients in total')
    print(f'There are {num_cov} covid patients')
    print(f'There are {num_admitted} admitted patients')
    print(f'{num_died} patients died')


### Expected outcome

---

<a name='1'></a>
## Part 2: Data Wrangling: set up the dataframe (30 pt)

In this part we are going to combine data to create a dataframe with values of interest for the lab values analysis. 

We would like a dataframe containing the following information per record (only Covid patients!!!)

- `PATIENT` - the ID of the covid patient
- `days` - the number of days the patient is under observation
- `CODE-Y` - the code of the observation  
- `VALUE` - the lab value of the observation

where only the following observation codes needs to be selected:

- `48065-7`  Fibrin D-dimer FEU [Mass/volume] in Platelet poor plasma
- `26881-3`   Interleukin 6 [Mass/volume] in Serum or Plasma
- `2276-4` Ferritin [Mass/volume] in Serum or Plasma
- `89579-7` Troponin I.cardiac [Mass/volume] in Serum or Plasma by High sensitivity method
- `731-0` Lymphocytes [#/volume] in Blood by Automated count
- `14804-9` Lactate dehydrogenase [Enzymatic activity/volume] in Serum or Plasma by Lactate to pyruvate reaction

The days information is not primarely available and needs to be calculated by substracting observation DATE - START. 

An example of such a dataframe is given below:

In [3]:
#Possible approach:

#Select all the patients with covid from the conditions table
#Combine conditions table (only covid patients) with the patient table into a covid_patient table
#select the only the relevant lab observations from the observations table into a lab_obs table
#merge the covid_patient table with the lab_obs table into a covid_patients_obs table
#clean the covid_patients_obs table (rename columns, select only relevant columns, sort, typecast, add days column)


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
    <ul><li>you can use pandas.DataFrame.merge() to merge dataframes</li>
    <li>df = df[(df.CODE == condition1 | df.CODE == condition1 )] selects rows with CODE of 2 conditional values</li>
    <li>df.DATE - df.START return days if DATE and START are datetime format</li>
    <li>pd.to_datetime() can be used to typecast to datetime</li>
</ul>
</details>

<a name='ex-21'></a>
### 2.1 Code your solution

In [4]:
#YOUR CODE HERE
df_covid_patient = pd.DataFrame(df_condition[df_condition['CODE']==840539006])
df_covid_patient=pd.merge(df_covid_patient, df_patient, left_on='PATIENT', right_on='Id')
df_lab_obs=df_observ.loc[(df_observ['CODE'] == '48065-7')|(df_observ['CODE'] =='26881-3')|(df_observ['CODE'] == '2276-4')|\
    (df_observ['CODE'] == '89579-7')|(df_observ['CODE'] == '731-0')|(df_observ['CODE'] == '14804-9')]
df_covid_patients_obs=pd.merge(df_covid_patient, df_lab_obs, on='PATIENT')
df_covid_patients_obs=df_covid_patients_obs.loc[:,['PATIENT','START','DATE','CODE_y','VALUE','UNITS']]
df_covid_patients_obs['days']=pd.to_datetime(df_covid_patients_obs['DATE'])- pd.to_datetime(df_covid_patients_obs['START'])
df_covid_patients_obs['days']=pd.to_numeric(df_covid_patients_obs['days'])/86400000000000
df_covid_patients_obs=df_covid_patients_obs.loc[:,['days','PATIENT','CODE_y','VALUE','UNITS']]
df_covid_patients_obs.set_index('PATIENT', inplace= True)
df_covid_patients_obs= df_covid_patients_obs.sort_index()
df_covid_patients_obs



,days,CODE_y,VALUE,UNITS
PATIENT,,,,
00079a57-24a8-430f-b4f8-a1cf34f90060,6.0,89579-7,2.3,pg/mL
00079a57-24a8-430f-b4f8-a1cf34f90060,6.0,2276-4,463.9,ug/L
00079a57-24a8-430f-b4f8-a1cf34f90060,6.0,48065-7,0.5,ug/mL
00079a57-24a8-430f-b4f8-a1cf34f90060,6.0,731-0,0.8,10*3/uL
00079a57-24a8-430f-b4f8-a1cf34f90060,5.0,731-0,1.0,10*3/uL
...,...,...,...,...
ffdbbb1b-745e-4e38-ade2-a19d6e778fee,6.0,14804-9,247.1,U/L
ffdbbb1b-745e-4e38-ade2-a19d6e778fee,7.0,731-0,0.9,10*3/uL
ffdbbb1b-745e-4e38-ade2-a19d6e778fee,8.0,731-0,0.8,10*3/uL


---

<a name='2'></a>
## Part 3: Data Wrangling, split into survived and not survived (10 pt)

Now we have the required data we would like to split the data into survived and not survived. First we fetch all the ids of the survived and deceased patients. We can use these ids to select the records of the survived patients and the patients that did not survived

Your job is to split the data into survived and not survived records. There are multiple ways to do this. One way is the  `.isin()` method

In [5]:
#the following code is given, RUN THIS CELL
#get survived and deceased ids
completed_isolation_patients = df_careplan[(df_careplan.CODE == 736376001) & (df_careplan.STOP.notna()) \
                                          & (df_careplan.REASONCODE == 840539006)].PATIENT
negative_covid_patient_ids = df_observ[(df_observ.CODE == '94531-1') \
                                          & (df_observ.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
survivor_ids = np.union1d(completed_isolation_patients, negative_covid_patient_ids)
deceased_ids = df_patient[df_patient.DEATHDATE.notna()].Id





<a name='ex-31'></a>
### 3.1 Code your solution

In [6]:
#YOUR CODE HERE

df_survived = df_covid_patients_obs[df_covid_patients_obs.index.isin(survivor_ids)]
df_died = df_covid_patients_obs[df_covid_patients_obs.index.isin(deceased_ids)]
len_survived=len(df_survived)
len_died = len(df_died)
print(len_survived, len_died)





57303 16793


### 3.2 Test your solution

In [7]:
def test3(survived, died):
    print(f'patients records survived: {survived}, patients records deceased {died}')
#call the test3
test3(survived=len_survived,died=len_died)

patients records survived: 57303, patients records deceased 16793


#### Expected outcome

---

<a name='3'></a>
## Part 4: Plot the data (20 pt)

Create plots with the lab data, for each code one plot. Separate the survivors and the deceased by color. An example of such a plot is given below. You can create 6 plots in one grid (for each code one plot) or use a widget (for instance a drop down menu widget) to select a lab CODE. Plot on the x-axis the days, on the y-axis the VALUE. Use proper labels, titles and legends.

<img src="../images/week3_plot.png" width="500" height="500"/>

<a name='ex-41'></a>
### 4.1 Code your solution

In [8]:
#YOUR CODE HERE
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.io import output_notebook

output_notebook()

def show_plot(code,survived,died):
    survived=pd.DataFrame(survived[survived['CODE_y']==code])
    died=pd.DataFrame(died[died['CODE_y']==code])
    fig=figure(width=300, height=300, title="The plot of "+code, x_axis_label='days',y_axis_label=' pg/ml')
    fig.scatter(x=survived['days'],y=survived['VALUE'], color='green',legend="survived")
    fig.scatter(x=died['days'],y=died['VALUE'], color='red',legend="deceased")
    fig.legend.location = 'top_left'
    
    return fig
fig1=show_plot(code='48065-7', survived=df_survived, died=df_died)
fig2=show_plot(code='26881-3', survived=df_survived, died=df_died)
fig3=show_plot(code='2276-4', survived=df_survived, died=df_died)
fig3.legend.location='top_right'
fig4=show_plot(code='89579-7', survived=df_survived, died=df_died)
fig4.legend.location='top_right'
fig5=show_plot(code='731-0', survived=df_survived, died=df_died)
fig6=show_plot(code='14804-9', survived=df_survived, died=df_died)
grid = gridplot(children = [[fig1,fig2],[fig3,fig4],[fig5,fig6]],plot_width=450, plot_height=450)
show(grid)


Loading BokehJS ...

<a name='4'></a>
## Part 5: Plot the location of the patients (10 pt)

This is a bonus part. Can you plot the patients location on a map? See also 
https://docs.bokeh.org/en/latest/docs/user_guide/geo.html


<a name='ex-51'></a>
### 5.1 Code your solution

In [9]:
#YOUR CODE HERE